<img src="https://i.pinimg.com/originals/ee/2e/24/ee2e246795a13abc1fe01b28776626ef.jpg" alt= "LOGO CAT" width=400 height=400 align = "right">

<br>
<h1><font color="#7F000E" size=5>INSTITUTO GEOFISICO DEL PERU</font></h1>
<h1><font color="#7F000R" size=6> SPARK Y PYTHON CON PYSPARK </font></h1>
<h1><font color="#7F000E" size=4> CUARTA PRUEBA Y EJEMPLOS
 </font></h1>
<br>
<br>
<div style="text-align:left">
<font color="#7F000E" size=3> Ing. Alexander Valdez</font><br>
<font color="#7F000E" size=3> Libertad Financiera </font><br>
<font color="#7F000e" size=3> Cuarta Clase </font><br>
</div>

- Apache Spark™ is a multi-language engine for executing data engineering, data science, and machine learning on single-node machines or clusters.
- MLlib is Apache Spark's scalable machine learning library. Ease of use. Usable in Java, Scala, Python, and R.

MLlib is Spark’s machine learning (ML) library. Its goal is to make practical machine learning scalable and easy. At a high level, it provides tools such as:
- 
ML Algorithms: common learning algorithms such as classification, regression, clustering, and collaborative filterin- g
Featurization: feature extraction, transformation, dimensionality reduction, and selecti- on
Pipelines: tools for constructing, evaluating, and tuning ML Pipeli- nes
Persistence: saving and load algorithms, models, and Pipel- ines
Utilities: linear algebra, statistics, data handling, etc.

Existen 2 tecnicas:
- DataFrame API Examples
- RDD API Examples

https://spark.apache.org/examples.html

# DATAFRAME API EXAMPLES
Veamos como resolver basicamente un caso de uso de aprendizaje automatico.
- Algoritmos de regresión

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LinearRegression').getOrCreate()

In [51]:
## READ THE DATASET
training = spark.read.csv('Book4.csv',header=True,inferSchema=True)

In [52]:
training.show()

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
|  Alex| 31|        10| 30000|
| Maria| 30|         8| 25000|
| Wendy| 29|         4| 20000|
|Silvia| 24|         3| 20000|
|   Paz| 21|         1| 15000|
|  Ciro| 23|         2| 18000|
+------+---+----------+------+



In [53]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [56]:
training.columns

['Name', 'age', 'Experience', 'Salary']

### USUALMENTE HACEMOS UN PREPROCESAMIENTO DE DATOS ANTES
[Age,Experience]---> new feature --> independent feature

In [57]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [58]:
output=featureassembler.transform(training)

In [59]:
output.show()

+------+---+----------+------+--------------------+
|  Name|age|Experience|Salary|Independent Features|
+------+---+----------+------+--------------------+
|  Alex| 31|        10| 30000|         [31.0,10.0]|
| Maria| 30|         8| 25000|          [30.0,8.0]|
| Wendy| 29|         4| 20000|          [29.0,4.0]|
|Silvia| 24|         3| 20000|          [24.0,3.0]|
|   Paz| 21|         1| 15000|          [21.0,1.0]|
|  Ciro| 23|         2| 18000|          [23.0,2.0]|
+------+---+----------+------+--------------------+



In [60]:
 output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [61]:
finalized_data= output.select("Independent Features","Salary")

In [62]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [63]:
  from pyspark.ml.regression import LinearRegression
### train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor= regressor.fit(train_data)

In [64]:
### Coefficients
regressor.coefficients

DenseVector([125.0, 1375.0])

In [65]:
### intercepts
regressor.intercept

12375.0

In [66]:
type(train_data)

pyspark.sql.dataframe.DataFrame

In [67]:
type(test_data)

pyspark.sql.dataframe.DataFrame

In [68]:
### prediction
from pyspark.ml.evaluation import RegressionEvaluator
pred_results = regressor.transform(test_data)

In [69]:
pred_results.show(3)

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16374.999999999996|
|          [24.0,3.0]| 20000|           19500.0|
|          [29.0,4.0]| 20000|           21500.0|
+--------------------+------+------------------+
only showing top 3 rows



In [76]:
rmse =  RegressionEvaluator(labelCol= 'Salary',predictionCol='prediction',metricName='rmse')
mae =  RegressionEvaluator(labelCol= 'Salary',predictionCol='prediction',metricName='mae')


In [75]:
rmse.evaluate(pred_results)

1492.1670482891643

In [77]:
mae.evaluate(pred_results)

1374.999999999999